In [31]:
# Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import pymongo
import json
import pprint

# Google developer API key
# from config import gkey
gkey = ''

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\walto\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


In [3]:
url = 'https://visitseattle.org/things-to-do/sightseeing/top-25-attractions/'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [27]:
a=soup.find_all('div', class_="medium-7")
attractions = []
for x in a:
    s= x.find('h3').text
    attractions.append(s)
# Close splinter window
browser.quit()

In [78]:
attraction_details = []

# base url
base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"

for attraction in attractions:
    # set up a parameters dictionary
    params = {
        "locationbias": "point:47.6062, 122.3321",
        "input": attraction,
        "inputtype":"textquery",
        "key": gkey,
        "fields":"name,geometry"
        }
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    # convert response to json
    places_data = response.json()
    print(places_data["candidates"][0]["name"])
    print(places_data["candidates"][0]["geometry"]["location"]['lat'])
    print(places_data["candidates"][0]["geometry"]["location"]['lng'])
    print("------------------")
    attraction_details.append({"name":places_data["candidates"][0]["name"],
                                "lat":places_data["candidates"][0]["geometry"]["location"]['lat'],
                                "lng":places_data["candidates"][0]["geometry"]["location"]['lng']})

Space Needle
47.6205063
-122.3492774
------------------
Seattle Center
47.6212504
-122.349689
------------------
Museum of Pop Culture
47.6214824
-122.3481245
------------------
Pacific Science Center
47.6189535
-122.351576
------------------
Chihuly Garden and Glass
47.6205599
-122.3504476
------------------
Pacific Northwest Ballet
47.6241852
-122.3514828
------------------
Pike Place Market
47.6096559
-122.3421521
------------------
Seattle Art Museum
47.6075253
-122.3379599
------------------
Central Public Library
1.2972003
103.8545058
------------------
Smith Tower
47.6018528
-122.3318583
------------------
Seattle Aquarium
47.6074002
-122.3429558
------------------
Anacortes Ferry Terminal
48.5067623
-122.6783634
------------------
The Seattle Great Wheel
47.6061342
-122.3425246
------------------
The Museum of Flight
47.5185379
-122.2968545
------------------
Woodinville Wine Country
47.7521171
-122.1584838
------------------
Boeing Future of Flight​
47.9212293
-122.2901597
---

In [79]:
attraction_details

[{'name': 'Space Needle', 'lat': 47.6205063, 'lng': -122.3492774},
 {'name': 'Seattle Center', 'lat': 47.6212504, 'lng': -122.349689},
 {'name': 'Museum of Pop Culture', 'lat': 47.6214824, 'lng': -122.3481245},
 {'name': 'Pacific Science Center', 'lat': 47.6189535, 'lng': -122.351576},
 {'name': 'Chihuly Garden and Glass', 'lat': 47.6205599, 'lng': -122.3504476},
 {'name': 'Pacific Northwest Ballet', 'lat': 47.6241852, 'lng': -122.3514828},
 {'name': 'Pike Place Market', 'lat': 47.6096559, 'lng': -122.3421521},
 {'name': 'Seattle Art Museum', 'lat': 47.6075253, 'lng': -122.3379599},
 {'name': 'Central Public Library', 'lat': 1.2972003, 'lng': 103.8545058},
 {'name': 'Smith Tower', 'lat': 47.6018528, 'lng': -122.3318583},
 {'name': 'Seattle Aquarium', 'lat': 47.6074002, 'lng': -122.3429558},
 {'name': 'Anacortes Ferry Terminal', 'lat': 48.5067623, 'lng': -122.6783634},
 {'name': 'The Seattle Great Wheel', 'lat': 47.6061342, 'lng': -122.3425246},
 {'name': 'The Museum of Flight', 'lat': 